### Installing Packages

In [1]:
# !pip install openai python-dotenv

### Loading Packages

In [2]:
import openai
from openai import OpenAI, AsyncOpenAI
import os

from dotenv import main
from IPython.display import Markdown

In [3]:
# loading openai api key
_ = main.load_dotenv(main.find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

## Using OpenAI

In [4]:
model_name = "gpt-4o"

In [5]:
# Initializing a client
client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)

def askGPT(question):
   response = client.chat.completions.create(model=model_name, 
                                             messages=[{ 
                                                         "role": "user", 
                                                         "content": f"{question}"
                                                      }])
   return response.choices[0].message.content

In [6]:
# Markdown(askGPT("Tell me about RAGs"))

### Lets Play

In [ ]:
askGPT("What is 1+1 ?, Just give me answer in single word in number format")

### Translating text

In [11]:
customer_email = """Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [12]:
style = """In American english with calm tone"""

In [13]:
prompt = f"""Translate the text that is delimited by triple backticks into a style that is {style}. \
    text: ```{customer_email}```
    """

In [14]:
response = askGPT(prompt)

In [ ]:
Markdown(response)

## Using LangChain

In [16]:
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(model_name = model_name)

In [ ]:
# List all default parameters and their values
default_params = vars(chat)
for param, value in default_params.items():
    print(f"{param}: {value}")

In [18]:
# changing temperature
chat = ChatOpenAI(temperature=0.0)

### Create a langchain prompt template

In [19]:
template_string = """Translate the text that is delimited by triple backticks into s style that is {style}. \
                    text : ```{text}``` 
                """

In [20]:
from langchain.prompts import ChatPromptTemplate

In [21]:
prompt_template = ChatPromptTemplate.from_template(template_string)

In [ ]:
prompt_template.messages[0].prompt

In [ ]:
prompt_template.messages[0].prompt.input_variables

In [24]:
customer_style = """American English \
in a calm and respectful tone
"""

In [25]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [26]:
customer_messages =  prompt_template.format_messages(
    style = customer_style,
    text = customer_email
)

In [27]:
customer_response = askGPT(customer_messages)

In [ ]:
Markdown(customer_response)

In [29]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [30]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [ ]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

Markdown(service_messages[0].content)

In [ ]:
service_response = askGPT(service_messages)
Markdown(service_response)

### Output parser

In [ ]:
# I want the output to look like below

{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

In [34]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

In [35]:
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

In [ ]:
messages = prompt_template.format_messages(text = customer_review)
response = askGPT(messages)
Markdown(response)

In [ ]:
type(response)

### Parse the LLM output string into a Python dictionary


In [39]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [40]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

In [42]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [ ]:
print(messages[0].content)

In [ ]:
response = askGPT(messages)
Markdown(response)

In [ ]:
response_json = output_parser.parse(response)
print(type(response_json))
print(response_json)